In [3]:
import argparse
import numpy as np
import pprint as pp
import pymunk
import pygame
import random
from tqdm.auto import tqdm
from utils import make_dir
import imageio

pygame 2.5.0 (SDL 2.28.0, Python 3.7.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
# Configuration
parser = argparse.ArgumentParser(description='Train EBM model')
parser.add_argument('--n_bodies', default=2, type=int,
                    help='Number of bodies')
parser.add_argument('--n_simulations', default=1000, type=int,
                    help='Number of simulations')
try:
    get_ipython().run_line_magic('matplotlib', 'inline')
    %load_ext autoreload
    %autoreload 2
    is_jupyter = True
    FLAGS = parser.parse_args([])
    FLAGS.n_bodies = 2
except:
    FLAGS = parser.parse_args()
pp.pprint(FLAGS.__dict__)

width, height = 200, 200
n_bodies = FLAGS.n_bodies
radius = 20
mass = 1
n_simulations = FLAGS.n_simulations
n_steps = 1000
is_visualize = True
filename = f'dataset/nbody_dataset/nbody-{n_bodies}/trajectory_balls_{n_bodies}_simu_{n_simulations}_steps_{n_steps}.npy'
videofile=f'dataset/nbody_dataset/nbody-{n_bodies}/trajectory_balls_{n_bodies}_simu_{n_simulations}_steps_{n_steps}.gif'
make_dir(filename)
print(f"Save file at {filename}.")

def add_body(space):
    inertia = pymunk.moment_for_circle(mass, 0, radius)
    body = pymunk.Body(mass, inertia)
    x = random.randint(radius, width - radius)
    y = random.randint(radius, height - radius)
    body.position = x, y

    # Assign a random initial velocity
    vx = random.uniform(-100, 100)
    vy = random.uniform(-100, 100)
    body.velocity = vx, vy

    shape = pymunk.Circle(body, radius)
    shape.elasticity = 1.0
    shape.friction = 0.0
    space.add(body, shape)
    return body

def add_walls(space):
    walls = [
        pymunk.Segment(space.static_body, (0, 0), (0, height), 1),
        pymunk.Segment(space.static_body, (0, height), (width, height), 1),
        pymunk.Segment(space.static_body, (width, height), (width, 0), 1),
        pymunk.Segment(space.static_body, (width, 0), (0, 0), 1)
    ]
    for wall in walls:
        wall.elasticity = 1.0
        wall.friction = 0.0
        space.add(wall)

def draw_bodies(screen, bodies):
    for body in bodies:
        x, y = body.position
        color = pygame.Color(body.color)
        pygame.draw.circle(screen, color, (int(x), int(y)), radius)

def run_simulation(space, bodies, n_steps, clock, screen=None):
    position_data = np.zeros((n_steps, len(bodies), 4))
    frames=[]
    for i in range(n_steps):
        if screen is not None:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    return None

        for j, body in enumerate(bodies):
            position_data[i, j, :2] = body.position
            position_data[i, j, 2:] = body.velocity

        if screen is not None:
            screen.fill((255, 255, 255))
            draw_bodies(screen, bodies)  # Draw the bodies with colors

        space.step(1/60.0)
        if screen is not None:
            pygame.display.flip()
        screen_data = pygame.surfarray.array3d(screen)
        frames.append(screen_data)
        clock.tick(60)
    imageio.mimsave(videofile, frames, 60)
    return position_data

def main():
    pygame.init()
    if is_visualize:
        screen = pygame.display.set_mode((width, height))
    else:
        screen = None
    clock = pygame.time.Clock()

    all_position_data = []

    for sim in tqdm(range(n_simulations)):
        print(f"simu: {sim}")
        space = pymunk.Space()
        space.gravity = (0, 0)

        add_walls(space)  # Add the walls

        bodies = [add_body(space) for _ in range(n_bodies)]

        # Assign a random color to each body
        for body in bodies:
            body.color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

        position_data = run_simulation(space, bodies, n_steps, clock, screen=screen)
        if position_data is not None:
            all_position_data.append(position_data)
        else:
            break
        
        # Save the position data to an .npy file
        all_position_data_save = np.array(all_position_data)
        np.save(filename, all_position_data_save) ##[n_simulations, n_steps, n_bodies, 4 ] 4 inculde (x,y,vx,vy)

    pygame.quit()

{'n_bodies': 2, 'n_simulations': 1000}
Save file at dataset/nbody_dataset/nbody-2/trajectory_balls_2_simu_1000_steps_1000.npy.


In [5]:
if __name__ == "__main__":
    main()

ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM default


  0%|          | 0/1000 [00:00<?, ?it/s]

simu: 0
simu: 1
simu: 2
simu: 3
simu: 4
simu: 5
simu: 6
simu: 7
simu: 8
simu: 9
simu: 10
simu: 11


KeyboardInterrupt: 

In [2]:
##The following is to process data with adding more features for the dataset,just for two bodies
import torch
import math
import numpy as np
width=200
height=200
filename = f'dataset/nbody_dataset/nbody-{2}/trajectory_balls_{2}_simu_{1000}_steps_{1000}_processed.npy'
dataset_init=np.load("/user/project/inverse_design/dataset/nbody_dataset/nbody-2/trajectory_balls_2_simu_1000_steps_1000.npy")
dataset_init_tensor=torch.tensor(dataset_init)
dataset_final=torch.ones((dataset_init_tensor.shape[0],dataset_init_tensor.shape[1],dataset_init_tensor.shape[2],11),dtype=float)
for i in range(dataset_final.shape[0]):
    for j in range(dataset_final.shape[1]):
        for k in range(dataset_final.shape[2]):
            dataset_final[i][j][k][0]=dataset_init_tensor[i][j][k][0]
            dataset_final[i][j][k][1]=dataset_init_tensor[i][j][k][1]
            dataset_final[i][j][k][2]=dataset_init_tensor[i][j][k][2]
            dataset_final[i][j][k][3]=dataset_init_tensor[i][j][k][3]
            dataset_final[i][j][k][4]=dataset_init_tensor[i][j][k][0]-dataset_init_tensor[i][j][1-k][0]
            dataset_final[i][j][k][5]=dataset_init_tensor[i][j][k][1]-dataset_init_tensor[i][j][1-k][1]
            #distance between body1 and body2
            dataset_final[i][j][k][6]=torch.sqrt((dataset_init_tensor[i][j][k][0]-dataset_init_tensor[i][j][1-k][0])*(dataset_init_tensor[i][j][k][0]-dataset_init_tensor[i][j][1-k][0])+(dataset_init_tensor[i][j][k][1]-dataset_init_tensor[i][j][1-k][1])*(dataset_init_tensor[i][j][k][1]-dataset_init_tensor[i][j][1-k][1]))
            # distance between body and left wall
            dataset_final[i][j][k][7]=torch.abs(dataset_init_tensor[i][j][k][0])
            # distance between body and up wall
            dataset_final[i][j][k][8]=torch.abs(dataset_init_tensor[i][j][k][1])
            # distance between body and right wall
            dataset_final[i][j][k][9]=torch.abs(width-dataset_init_tensor[i][j][k][0])
            # distance between body and down wall
            dataset_final[i][j][k][10]=torch.abs(height-dataset_init_tensor[i][j][k][1])
dataset_final_np=np.array(dataset_final)
np.save(filename,dataset_final_np)
